<a href="https://colab.research.google.com/github/kapamawi/AI/blob/main/2_6_1___rnw_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
!pip install -qU crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29 openai

Ten kod instaluje kilka bibliotek Pythona za pomocą menedżera pakietów pip:

`crewai` w wersji 0.28.8 - biblioteka służąca do tworzenia i zarządzania zespołami agentów AI

`crewai_tools` w wersji 0.1.6 - zestaw narzędzi rozszerzających funkcjonalność biblioteki crewai

`langchain_community` w wersji 0.0.29 - biblioteka zawierająca komponenty do budowania aplikacji opartych na modelach językowych

`openai` - oficjalny pakiet OpenAI umożliwiający integrację z ich modelami AI

Flaga `-q` oznacza tryb "quiet" (cichy), który ogranicza wyświetlanie komunikatów podczas instalacji.

Flaga `-U` (lub `--upgrade`) oznacza, że jeśli któraś z tych bibliotek jest już zainstalowana, zostanie zaktualizowana do podanej wersji.

In [ ]:
import os
from crewai import Agent, Task, Crew, Process
from openai import OpenAI
import time
import pandas as pd
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain.agents import Tool
from google.colab import userdata

Ten kod importuje niezbędne biblioteki i komponenty:

`os` - moduł dostarczający funkcje do interakcji z systemem operacyjnym

Z biblioteki `crewai` importowane są cztery główne klasy:
- `Agent` - do tworzenia agentów AI
- `Task` - do definiowania zadań
- `Crew` - do organizowania zespołów agentów
- `Process` - do zarządzania procesami

`OpenAI` - klasa klienta API OpenAI służąca do komunikacji z modelami OpenAI

`time` - moduł zawierający funkcje związane z czasem

`pandas` (jako `pd`) - biblioteka do analizy i manipulacji danymi

Z pakietu `langchain_community.document_loaders` importowany jest `CSVLoader` - narzędzie do wczytywania plików CSV

Z `langchain.agents` importowana jest klasa `Tool` służąca do tworzenia narzędzi dla agentów

`userdata` z `google.colab` pozwala na dostęp do danych użytkownika w środowisku Google Colab

In [ ]:
class CFG:
    model = 'gpt-4o-mini'

In [ ]:

os.environ["OPENAI_API_KEY"] = userdata.get('openaivision')


# Dane

In [ ]:
df = pd.read_csv('/content/telecom_churn.csv')
df.head(10)

In [ ]:
df[['Churn', 'AccountWeeks', 'DataUsage','ContractRenewal', 'RoamMins']].sample(100).reset_index(drop = True).to_csv('sub_telecom_churn.csv', index = False)

Ten kod wykonuje kilka operacji na ramce danych (DataFrame) o nazwie `df`:

1. Wybiera tylko określone kolumny z danych:
   - `Churn` (odejścia klientów)
   - `AccountWeeks` (liczba tygodni posiadania konta)
   - `DataUsage` (zużycie danych)
   - `ContractRenewal` (odnowienie umowy)
   - `RoamMins` (minuty w roamingu)

2. Za pomocą `sample(100)` losowo wybiera 100 wierszy z tych danych

3. `reset_index(drop = True)` resetuje indeksy wybranych wierszy, a parametr `drop = True` usuwa starą kolumnę indeksów

4. `to_csv('sub_telecom_churn.csv', index = False)` zapisuje wynikowe dane do pliku CSV o nazwie 'sub_telecom_churn.csv'. Parametr `index = False` oznacza, że indeksy wierszy nie zostaną zapisane w pliku

Jest to typowa operacja przygotowania mniejszego zbioru danych do testów lub analizy, w tym przypadku z danych telekomunikacyjnych dotyczących odejść klientów.

# Agentura

In [ ]:
# tool = data loader
loader = CSVLoader(file_path="sub_telecom_churn.csv")

load_tool = Tool(
  name="Data Loader Tool",
  func=loader.load,
  description="Load the CSV data from the source",
)

Ten fragment kodu tworzy narzędzie do wczytywania danych:

`loader = CSVLoader(file_path="sub_telecom_churn.csv")` - tworzy obiekt do wczytywania danych z pliku CSV 'sub_telecom_churn.csv'

Następnie definiowane jest narzędzie `load_tool` za pomocą klasy `Tool` z trzema parametrami:
- `name` - nazwa narzędzia: "Data Loader Tool"
- `func` - funkcja wykonywana przez narzędzie: `loader.load` (wczytywanie danych)
- `description` - opis działania narzędzia: "Load the CSV data from the source"

To narzędzie będzie używane przez agentów AI do wczytywania i analizowania danych z pliku CSV.

In [ ]:
analyst = Agent(
  role='Senior Data Analyst',
  goal='Load the dataset and provide insights from the data',
  backstory="""You work at a telecom company.
  Your expertise lies in identifying trends and analysis from the available data.
  You have a knack for dissecting complex data and presenting
  actionable insights.""",
  verbose=True,
  allow_delegation=False,
)

writer = Agent(
  role='Data Content Strategist',
  goal='Craft compelling content from the data',
  backstory="""You are a renowned data writer, known for
  your insightful and engaging articles from the dataset.
  You transform complex concepts into compelling narratives.""",
  verbose=True,
  allow_delegation=False,
)

Ten kod tworzy dwóch agentów AI:

Pierwszy agent - `analyst`:
- Rola: Starszy Analityk Danych
- Cel: Wczytanie danych i dostarczenie wniosków
- Historia: Pracuje w firmie telekomunikacyjnej, specjalizuje się w identyfikacji trendów i analizie dostępnych danych, posiada umiejętność rozbioru złożonych danych i przedstawiania praktycznych wniosków
- `verbose=True` - będzie pokazywał szczegółowe informacje o swoich działaniach
- `allow_delegation=False` - nie może delegować zadań innym agentom

Drugi agent - `writer`:
- Rola: Strateg Treści Danych
- Cel: Tworzenie przekonujących treści na podstawie danych
- Historia: Znany autor tekstów o danych, tworzy wnikliwe i wciągające artykuły na podstawie zbiorów danych, przekształca złożone koncepcje w przekonujące narracje
- `verbose=True` - będzie pokazywał szczegółowe informacje o swoich działaniach
- `allow_delegation=False` - nie może delegować zadań innym agentom

In [ ]:
task1 = Task(
  description="""Conduct a comprehensive analysis from the CSV dataset by using the tools provided.
  You only need to run the tools and without providing any input, as the tools already have the source data path embedded to it.
  Identify any analysis that related to why churn is happening.
  Your final answer MUST be a full analysis report""",
  agent=analyst,
  tools = [load_tool],
  expected_output = "Full analysis report"
)


task2 = Task(
  description="""Using the insights provided, develop an engaging blog
  post that highlight your finding for the others.
  Your post should be informative, especially for casual people.
  Make it profoessional and avoid complex words so it doesn't sound like AI.
  Your final answer MUST be the full blog post of at least 3 paragraphs.""",
  agent=writer,
    expected_output = "A text of the blog post"
)

Ten kod definiuje dwa zadania dla wcześniej utworzonych agentów:

`task1` - zadanie dla analityka:
- Opis: przeprowadzenie kompleksowej analizy danych z pliku CSV przy użyciu dostarczonych narzędzi. Narzędzia mają już zdefiniowaną ścieżkę do danych źródłowych, więc nie wymagają dodatkowego wprowadzania danych. Zadaniem jest zidentyfikowanie przyczyn rezygnacji klientów (churn)
- Przypisany agent: `analyst`
- Dostępne narzędzia: `load_tool`
- Oczekiwany wynik: pełny raport z analizy

`task2` - zadanie dla pisarza:
- Opis: stworzenie wciągającego wpisu na blog na podstawie dostarczonych wniosków. Tekst ma być informacyjny i zrozumiały dla przeciętnego czytelnika. Ma być profesjonalny, ale bez skomplikowanego słownictwa, aby nie brzmiał jak tekst AI. Wymagane minimum 3 akapity
- Przypisany agent: `writer`
- Oczekiwany wynik: gotowy tekst wpisu na blog

In [ ]:
crew = Crew(
  agents=[analyst, writer],  tasks=[task1, task2],
  verbose= True,
)

result = crew.kickoff()

 [DEBUG]: == Working Agent: Senior Data Analyst
 [INFO]: == Starting Task: Conduct a comprehensive analysis from the CSV dataset by using the tools provided.
  You only need to run the tools and without providing any input, as the tools already have the source data path embedded to it.
  Identify any analysis that related to why churn is happening.
  Your final answer MUST be a full analysis report


> Entering new CrewAgentExecutor chain...
I need to load the CSV data from the source using the Data Loader Tool to start the analysis. 

Action: 
Data Loader Tool

Action Input: 
{}
 

[Document(page_content='Churn: 0\nAccountWeeks: 127\nDataUsage: 3.13\nContractRenewal: 1\nRoamMins: 11.6', metadata={'source': 'sub_telecom_churn.csv', 'row': 0}), Document(page_content='Churn: 1\nAccountWeeks: 68\nDataUsage: 3.83\nContractRenewal: 0\nRoamMins: 14.2', metadata={'source': 'sub_telecom_churn.csv', 'row': 1}), Document(page_content='Churn: 0\nAccountWeeks: 105\nDataUsage: 0.33\nContractRenewal

Ten kod tworzy i uruchamia zespół (crew) agentów AI:

`crew = Crew()` przyjmuje trzy parametry:
- `agents=[analyst, writer]` - lista agentów w zespole
- `tasks=[task1, task2]` - lista zadań do wykonania
- `verbose=True` - włącza szczegółowe raportowanie działań

`result = crew.kickoff()` uruchamia pracę zespołu:
1. Najpierw analityk przeprowadzi analizę danych z pliku CSV
2. Następnie pisarz wykorzysta wyniki tej analizy do stworzenia wpisu na blog
3. Wyniki zostaną zapisane w zmiennej `result`

Kolejność zadań jest istotna - drugie zadanie (pisanie wpisu) jest zależne od wyników pierwszego (analizy danych).